In [1]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import pyperclip as pc
import random
import os
from docx import Document
import re
from IPython.display import clear_output

In [2]:
def get_chrome_driver_with_profile():
    #profile path
    profile_path = r'C:\Users\gsd463\AppData\Local\Google\Chrome\User Data'
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f'user-data-dir={profile_path}')
    #provide the profile name with which we want to open browser
    chrome_options.add_argument('--profile-directory=Profile 1')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver

# AUTO POST CHAPTER FOR ALL WEBSITE SUPPORTED

In [6]:
# Hàm auto post truyện chương bằng Selenium
def open_and_post_chapter_vlog(driver, first_url, start_chapter, number_of_chapter = 2):
    count = 0
    while(count<number_of_chapter):
        try:
            driver.get(first_url)
            stt_chapter = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/form/div[2]/input')
            starting_chapter = stt_chapter.get_attribute("value")
            if(count == 0):
                start_chapter = starting_chapter
            file_path = f'G:\\My Drive\\Sword God in a World of Magic\\ch{starting_chapter}.docx'
            first_sentence, all_content = get_first_sentence(file_path)
            first_key, first_value = split_string(first_sentence)
            print(first_key + " " + first_value)
            print(all_content)
            pc.copy(all_content)
            # Tìm text box number chương để nhập vào
            chapter_stt_input = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/form/div[1]/input')
            if(chapter_stt_input):
                chapter_stt_input.clear()
                chapter_stt_input.send_keys(starting_chapter)
            chapter_header_text_input = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/form/div[3]/input')
            if(chapter_header_text_input):
                chapter_header_text_input.clear()
                chapter_header_text_input.send_keys(first_sentence)
            time.sleep(1)
            vip_click = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/form/div[6]/input')
            if(vip_click):
                vip_click.click()
            input_money = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/form/div[7]/input')
            if(input_money):
                input_money.clear()
                input_money.send_keys(350)
            time.sleep(2)
            submit_button = driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div/div[2]/form/button')
            if(submit_button):
                submit_button.click()
            time.sleep(3)
            textarea = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/form/div[12]/div/div/div')
            if(textarea):
                textarea.click()
                active_ele = driver.switch_to.active_element
                active_ele.send_keys(Keys.CONTROL, 'v')

            button_save = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/form/button')
            if(button_save):
                button_save.click()
                time.sleep(2)
                print(f'Posted chapter {starting_chapter}')
        except ConnectionRefusedError:
            print(f'ConnectionRefusedError at chapter {starting_chapter}')
        count = count+1
    #gọi qua trang truyenhd để đăng
    open_and_post_chapter_truyenhdx(driver, truyenhdx_url, start_chapter)

def open_and_post_chapter_truyenhdx(driver, url,start_chapter, number_of_chapter = 2):
    count = 0
    while(count<number_of_chapter):
        try:
            driver.get(url)
            starting_chapter = start_chapter
            file_path = f'G:\\My Drive\\Sword God in a World of Magic\\ch{starting_chapter}.docx'
            first_sentence, all_content = get_first_sentence(file_path)
            first_key, first_value = split_string(first_sentence)
            print(first_key + " " + first_value)
            print(all_content)
            pc.copy(all_content)
            # Tìm text box number chương để nhập vào
            chapter_header_text_input = driver.find_element(By.XPATH,'/html/body/div[5]/div/div[2]/div[5]/div/input')
            if(chapter_header_text_input):
                chapter_header_text_input.clear()
                chapter_header_text_input.send_keys(first_sentence)
            time.sleep(1)
            textarea = driver.find_element(By.XPATH, '/html/body/div[5]/div/div[2]/div[6]/div/div[2]')
            if(textarea):
                textarea.click()
                active_ele = driver.switch_to.active_element
                active_ele.send_keys(Keys.CONTROL, 'v')
            time.sleep(1)
            dropdown = Select(driver.find_element(By.ID,'mission_type_price'))
            if(dropdown):
                dropdown.select_by_value('1')
                time.sleep(1)
            fee_input = driver.find_element(By.XPATH, '/html/body/div[5]/div/div[2]/div[7]/div[2]/div[1]/input')
            if(fee_input):
                fee_input.send_keys(400)
            time.sleep(3)
            button_save = driver.find_element(By.ID, 'them_chuong')
            if(button_save):
                button_save.click()
                time.sleep(2)
                print(f'Posted chapter {starting_chapter}')
        except ConnectionRefusedError:
            print(f'ConnectionRefusedError at chapter {starting_chapter}')
        count = count+1
        starting_chapter +=1
    #Gọi qua bns để đăng
    open_and_post_chapter_bns(driver, bns_url, start_chapter)

def open_and_post_chapter_bns(driver, url,start_chapter, number_of_chapter = 2):
    count = 0
    try:
        driver.get(url)
        starting_chapter = start_chapter
        while(count<number_of_chapter):
            file_path = f'G:\\My Drive\\Sword God in a World of Magic\\ch{starting_chapter}.docx'
            all_content = get_all_content_word(file_path)
            print(all_content)
            pc.copy(all_content)
            textarea = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div/div/form/div[1]/textarea')
            if(textarea):
                textarea.click()
                active_ele = driver.switch_to.active_element
                active_ele.send_keys(Keys.PAGE_DOWN)
                active_ele.send_keys(Keys.CONTROL, 'v')
                active_ele.send_keys(Keys.ENTER)
            time.sleep(2)
            count = count+1
            starting_chapter +=1
        time.sleep(1)
        button_save = driver.find_element(By.XPATH, '/html/body/div/div/div/div[1]/div/div/form/div[3]/button[1]')
        if(button_save):
            button_save.click()
            time.sleep(2)
            print(f'Posted chapter {starting_chapter}')
    except ConnectionRefusedError:
        print(f'ConnectionRefusedError at chapter {starting_chapter}')


def get_first_sentence(file_path):
    # Đọc file word
        doc = Document(file_path)
        # Lấy dòng đầu tiên ra
        first_paragraph = doc.paragraphs[0].text
        all_content = doc.paragraphs[1:]
        all_text = '\n'.join([paragraph.text for paragraph in all_content])
        return first_paragraph, all_text

def get_all_content_word(file_path):
    # Đọc file word
        doc = Document(file_path)
        # Lấy dòng đầu tiên ra
        all_content = doc.paragraphs[0:]
        all_text = '\n'.join([paragraph.text for paragraph in all_content])
        return all_text

def split_string(chapter_header):
    # Cắt chuỗi dựa trên dấu hai chấm ':'
    split_parts = chapter_header.split(':')
    # Extract the two parts
    first_part = split_parts[0].strip()
    second_part = split_parts[1].strip()
    return first_part, second_part

# Set the website URL
vlognove_url = "https://vlognovel3.com/thanh-vien/chapter/658197d44315424836032496/add"
truyenhdx_url = "https://truyenhdx.com/user/quan-ly-truyen/them-chuong/?id=10323547#h2"
bns_url = "https://bachngocsach.vip/them-nhieu-chuong/than-kiem-luu-lac-phap-gioi-3/904.html"

d = get_chrome_driver_with_profile()
open_and_post_chapter_vlog(d, vlognove_url)

Chương 117: Giây phút cuối cùng
Shang buông cái đuôi ra.
ĐÙNG!
Cái đuôi quất mạnh xuống đất, phá hủy tan tát mặt đất bên dưới. Nếu Shang mà vẫn bám lấy cái đuôi thì chắc chắn xương sống của anh đã bị gãy. Vào lúc đó, anh sẽ nằm chết dí ở đó mà chờ chết.
Khi cái đuôi đập xuống đất, một vết máu nhỏ xuất hiện bên dưới nó. Shang đã hấp thụ một lượng lớn sinh lực từ đuôi con rắn, khiến cái đuôi nó yếu đi. Vì lẽ đó, nó không thể chịu được lực đập mạnh như thế.
Lúc này, cánh tay trái của Shang phù lên vì sinh lực tràn trề, anh cảm thấy như nó sắp nổ tung tới nơi vậy. Tuy nhiên, điều đó không thành vấn đề.
Đúng như dự đoán, đầu con rắn bắn về phía Shang. Sẽ rất khó để chặn đòn tấn công đó bằng thanh kiếm với vị trí hiện tại, nhưng dù sao thì anh cũng không có ý định làm thế.
Cánh tay của Shang hướng xuống dưới.
ĐÙNG! ĐÙNG!
Shang liên tiếp thi triển Hoả Bộc Phá và Hàn Bộc Phá. Lực đẩy ngược của hai vụ nổ đã đẩy anh lên cao vài mét, và con rắn đã tấn công hụt mục tiêu.
Cánh tay của Shang gần như